In [1]:
pip install jupyter-dash

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, dash_table
import dash_leaflet as dl
import pandas as pd
from crud_module import CRUD

# Connect to MongoDB
crud = CRUD("aacuser", "AACuser2025")
data = crud.read({})

# Convert MongoDB ObjectId to string so Dash can serialize it
df = pd.DataFrame(data)
df['_id'] = df['_id'].astype(str)
df.columns = df.columns.astype(str)

In [3]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3("Grazioso Salvare Dashboard – Unique ID: Rowe_DBC"),

    html.Div([
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict('records'),
            page_size=5,
            style_table={'overflowX': 'auto'},
            row_selectable="single",
            selected_rows=[0]
        )
    ]),

    html.Div(id='map-id', className='col s12 m6')
])

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[
                dff.iloc[row, 14], dff.iloc[row, 15]
            ],
            children=[
                dl.Tooltip(dff.iloc[row, 5]),
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row, 10])])
            ])
        ])
    ]


In [4]:
app.run_server(mode='inline', debug=True, port=8052)

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
File /usr/local/anaconda/lib/python3.9/site-packages/pandas/core/indexing.py:960, in _LocationIndexer.__getitem__(
    self=<pandas.core.indexing._iLocIndexer object>,
    key=(0, 14)
)
    958     key = tuple(com.apply_if_callable(x, self.obj) for x in key)
    959     if self._is_scalar_access(key):
--> 960         return self.obj._get_value(*key, takeable=self._takeable)
        key = (0, 14)
        self = <pandas.core.indexing._iLocIndexer object at 0x7f262a32ff40>
        self._takeable = True
    961     return self._getitem_tuple(key)
    962 else:
    963     # we by definition only have the 0th axis

File /usr/local/anaconda/lib/python3.9/site-packages/pandas/core/frame.py:3612, in DataFrame._get_value(
    self=Empty DataFrame
Columns: []
Index: [],
    index=0,
    col=14,
    takeable=True
)
   3593 """
   3594 Quickly retr